# 1D persistent bump attractor network

- persistent means that the firing activities run even if the stimulus lasts few instants (aka, there's no decay...)

In [1]:
# fileName

fileName = 'eg001_1D_persistent_bump-attractor'

In [2]:
# libs

import pyNN.spiNNaker as sim
import numpy as np
import matplotlib.pyplot as plt

Detected PyNN version 0.9.4 and Neo version 0.6.1


In [3]:
# simulation settings

sim.setup(
        timestep=1, # [ms]
        min_delay=1, # [ms]
        max_delay=100) # [ms]

simtime = 500 #ms

2021-04-10 15:02:42 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2021-04-10 15:02:42 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2021-04-10 15:02:42 INFO: Setting time scale factor to 1.
2021-04-10 15:02:42 INFO: Setting machine time step to 1000 micro-seconds.


['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


In [4]:
# defining the network

pops = {}
n_cells = {}
neuron_Model = {}
neuron_ModelPars = {}
initial_Values = {}

for popName in ['Exc', 'Inh']:
    n_cells[popName] = 100
    neuron_Model[popName] = sim.IF_cond_exp
    neuron_ModelPars[popName] = sim.IF_cond_exp.default_parameters # or standard_pars or testing_pars
    initial_Values[popName] = sim.IF_cond_exp.default_initial_values
    pops[popName] = sim.Population(n_cells[popName], 
                                   neuron_Model[popName],
                                   neuron_ModelPars[popName],
                                   label = popName)
                                   # add spatial constraint

    pops[popName].record(['spikes','v','gsyn_exc','gsyn_inh'])   
 
pops

{'Exc': <spynnaker8.models.populations.population.Population at 0x7f94d8664a08>,
 'Inh': <spynnaker8.models.populations.population.Population at 0x7f94d8664ad8>}

In [5]:
# defining the list of connections

def compute_manual_list(n_cells_i, n_cells_j, weights, d_thresh, p_thresh, width): 
        v_c = 0.35 #m/s
        d0 = 1 #ms
        scale = 1

        connections = {}
        probabilities = {}
        distances = {}

        connections = []
        probabilities = []
        distances = []
        for pre in range(n_cells_i):
            for post in range(n_cells_j):
                d_ij = np.sqrt((pre - post)**2)
                delay = d0 + d_ij / v_c
                if d_ij > d_thresh: 
                    distances.append(d_ij)
                    p_ij = scale*np.exp(-0.5 * (d_ij**2/width**2))
                    probabilities.append(p_ij)

                    if p_ij > p_thresh:
                        connections.append([pre, post, weights, delay])#, [d_ij, p_ij]])

        return connections, distances, probabilities


In [6]:
# compute manual list of connections to have center-surround structure

weight = {('Exc', 'Exc') : 0.08,
          ('Exc', 'Inh') : 0.08,
          ('Inh', 'Exc') : 0.24
          }

d_thresh = {('Exc', 'Exc'): 0, 
            ('Inh', 'Exc'): 2} 

p_thresh = {('Exc', 'Exc'): 0.1, 
            ('Inh', 'Exc'): 0.1}

width = {('Exc', 'Exc'): 1, 
         ('Inh', 'Exc'): 3}



connections = {}
distances = {}
probabilities = {}

connections['Exc', 'Exc'], distances['Exc', 'Exc'], probabilities['Exc', 'Exc'] = compute_manual_list(n_cells['Exc'], 
                                                                                                      n_cells['Exc'], 
                                                                                                      weight['Exc', 'Exc'], 
                                                                                                      d_thresh['Exc', 'Exc'], 
                                                                                                      p_thresh['Exc', 'Exc'], 
                                                                                                      width['Exc', 'Exc'])

connections['Inh', 'Exc'], distances['Inh', 'Exc'], probabilities['Inh', 'Exc'] = compute_manual_list(n_cells['Inh'], 
                                                                                                      n_cells['Exc'], 
                                                                                                      weight['Inh', 'Exc'], 
                                                                                                      d_thresh['Inh', 'Exc'], 
                                                                                                      p_thresh['Inh', 'Exc'], 
                                                                                                      width['Inh', 'Exc'])

In [20]:
# visual check
fig, ax = plt.subplots(1,2, figsize=(9,5))
fig.tight_layout(pad=3)
axes_list = fig.axes

axes_list[0].plot(np.asarray(connections['Exc', 'Exc']).T[0], np.asarray(connections['Exc', 'Exc']).T[1],'go', label='Exc (i) - Exc (j)')
axes_list[0].plot(np.asarray(connections['Inh', 'Exc']).T[0], np.asarray(connections['Inh', 'Exc']).T[1],'r+', label='Inh (i) - Exc (j)')
axes_list[0].grid()
axes_list[0].legend()
axes_list[0].set_title('scatter plot of connections')
axes_list[0].set_xlabel('i cells')
axes_list[0].set_ylabel('j cells')
axes_list[0].set_xlim(20,40)
axes_list[0].set_ylim(20,40)

axes_list[1].plot(distances['Exc', 'Exc'],probabilities['Exc', 'Exc'],'g+')
axes_list[1].plot(distances['Inh', 'Exc'],probabilities['Inh', 'Exc'],'r+') 
axes_list[1].plot(np.arange(0, 50), 1*np.exp(-0.5 * (np.arange(0, 50)**2/width['Exc', 'Exc']**2)), 'g:')
axes_list[1].plot(np.arange(0, 50), 1*np.exp(-0.5 * (np.arange(0, 50)**2/width['Inh', 'Exc']**2)), 'r:')
axes_list[1].grid()
axes_list[1].set_xlim(0,50)
axes_list[1].set_ylim(0,1)
axes_list[1].axhline(p_thresh['Inh', 'Exc'], color='k', label='p_threshold', )
axes_list[1].set_title('probability as function of distance')
axes_list[1].set_xlabel('distance')
axes_list[1].set_ylabel('probability')
axes_list[1].legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# make the projections

proj = {}



proj['Exc', 'Exc'] = sim.Projection(pops['Exc'], 
                                    pops['Exc'],
                                    connector = sim.FromListConnector(connections['Exc', 'Exc']),
                                    #synapse_type = sim.StaticSynapse(weight=0.08, delay=1),
                                    receptor_type = 'excitatory',
                                    #space=<pyNN.space.Space object at 0x7ff8f25a2110>,
                                    label = 'exc-exc connections'
                                    )

proj['Exc', 'Inh'] = sim.Projection(pops['Exc'], pops['Inh'],
                                    connector = sim.OneToOneConnector(), 
                                    synapse_type = sim.StaticSynapse(weight=weight['Exc', 'Inh'], delay=1),
                                    receptor_type = 'excitatory',
                                    #space=<pyNN.space.Space object at 0x7ff8f25a2110>,
                                    label = 'exc-inh connections'
                                    )


proj['Inh', 'Exc'] = sim.Projection(pops['Inh'], pops['Exc'],
                                    connector = sim.FromListConnector(connections['Inh', 'Exc']),
                                    #synapse_type = sim.StaticSynapse(weight=weight['Inh', 'Inh'], delay=1),
                                    receptor_type = 'inhibitory',
                                    #space=<pyNN.space.Space object at 0x7ff8f25a2110>,
                                    label = 'inh-exc connections'
                                    )

In [9]:
# make the thalamic input sources

popName = 'Thalamus'  

n_cells[popName] = n_cells['Exc']
spike_times = [[]]*n_cells[popName] #list of spike lists, where one spike list is related to one spike source
random_sources_idx = [np.random.randint(n_cells[popName]*0.40, n_cells[popName]*0.60) for i in range(n_cells[popName])]

for idx, sources in enumerate(random_sources_idx):
    spike_times[sources] = [abs(np.random.normal(loc=100, scale=10)) for n in range(10)]


neuron_Model[popName] = sim.SpikeSourceArray(spike_times)


pops[popName] = sim.Population(n_cells[popName], 
                           neuron_Model[popName],
                           label = popName)

proj['Thalamus', 'Exc'] = sim.Projection(pops['Thalamus'], 
                                         pops['Exc'],
                                         connector = sim.OneToOneConnector(),
                                         synapse_type = sim.StaticSynapse(weight=0.08, delay=1),
                                         receptor_type = 'excitatory',
                                         label = 'thalamus-exc connections')

fig, axes = plt.subplots(1,1)
axes_list = fig.axes
axes_list[0].eventplot(spike_times)
axes_list[0].set_xlabel('[ms]')
axes_list[0].set_ylabel('thalamic spike sources')
axes_list[0].set_title('rasterplot')
axes_list[0].set_xlim(0, simtime)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 500.0)

In [10]:
# run simulation

sim.run(simtime) # simtime=500 [ms]

2021-04-10 15:02:57 INFO: Simulating for 500 1.0ms timesteps using a hardware timestep of 1000us
2021-04-10 15:02:57 INFO: Starting execution process
2021-04-10 15:03:01 INFO: Time 0:00:03.884906 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-04-10 15:03:13 INFO: Time 0:00:11.909337 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-04-10 15:03:19 INFO: Time 0:00:05.730142 taken by PartitionAndPlacePartitioner
Created spalloc job 5927774
2021-04-10 15:03:19 INFO: Created spalloc job 5927774
Waiting for board power commands to complete.
2021-04-10 15:03:19 INFO: Waiting for board power commands to complete.
2021-04-10 15:03:24 INFO: Time 0:00:05.048797 taken by SpallocAllocator
2021-04-10 1

500.0

In [11]:
# save the results
outputs = {}

for layer in ['Exc', 'Inh']:
    outputs[layer] = pops[layer].get_data()
    for recording in ['v', 'gsyn_inh', 'gsyn_exc', 'spikes']:
        pops[layer].write_data(fileName + '_' + str(layer) + '_' + str(recording) + '.pkl')

Getting spikes for Exc
|0%                          50%                         100%|
Getting v for Exc
|0%                          50%                         100%|
Getting gsyn_exc for Exc
|0%                          50%                         100%|
Getting gsyn_inh for Exc
|0%                          50%                         100%|
Getting spikes for Exc
|0%                          50%                         100%|
Getting v for Exc
|0%                          50%                         100%|
Getting gsyn_exc for Exc
|0%                          50%                         100%|
Getting gsyn_inh for Exc
|0%                          50%                         100%|
Getting spikes for Exc
|0%                          50%                         100%|
Getting v for Exc
|0%                          50%                         100%|
Getting gsyn_exc for Exc
|0%                          50%                         100%|
Getting gsyn_inh for Exc
|0%                          50%  

In [12]:
# end simulation

sim.end()

2021-04-10 15:03:58 WARNING: During the extraction of data of 202000 bytes from memory address 1715844284, attempt 0 had 38 sequences that were lost.
2021-04-10 15:03:58 WARNING: During the extraction of data of 202000 bytes from memory address 1696192884, attempt 0 had 35 sequences that were lost.


In [13]:
def recover_results(outputs):
    results = {}
    for key in outputs.keys(): # to extract the name of the layer, e.g., Exc, Inh, Thalamus, etc  
        
        # to get voltage and conductances
        for analogsignal in outputs[key].segments[0].analogsignals:
            print(analogsignal.name)
            results[key, analogsignal.name] = analogsignal

        # to get spikes
        results[key, 'spikes'] = outputs[key].segments[0].spiketrains
    return results

In [14]:
# check results

results = recover_results(outputs)
results.keys()

v
gsyn_exc
gsyn_inh
v
gsyn_exc
gsyn_inh


dict_keys([('Exc', 'v'), ('Exc', 'gsyn_exc'), ('Exc', 'gsyn_inh'), ('Exc', 'spikes'), ('Inh', 'v'), ('Inh', 'gsyn_exc'), ('Inh', 'gsyn_inh'), ('Inh', 'spikes')])

In [15]:
# check the spikes

fig, axes = plt.subplots(2, 1)
fig.tight_layout(pad=5)
axes_list = fig.axes

for idx, value in enumerate(['Exc', 'Inh']):
    axes_list[idx].eventplot(results[value, 'spikes'])
    axes_list[idx].set_title('rasterplot of ' + str(value) + ' layer')
    axes_list[idx].set_xlabel('[ms]')
    axes_list[idx].set_ylabel('cells')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# check the voltage

fig, axes = plt.subplots(2, 1, figsize=(9,5))
fig.tight_layout(pad=5)

axes_list = fig.axes

for idx, value in enumerate(['Exc', 'Inh']):
    im = axes_list[idx].imshow(results[value, 'v'].T)
    axes_list[idx].set_title('voltage of ' + str(value) + ' layer')
    axes_list[idx].set_xlabel('time [ms]')
    axes_list[idx].set_ylabel('cells')
    fig.colorbar(im, ax=axes_list[idx], fraction=0.010, label='[mV]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# check the conductances

for layer in ['Exc', 'Inh']:
    fig, axes = plt.subplots(2, 1)
    fig.tight_layout(pad=5)
    fig.suptitle(str(layer) + ' layer')
    axes_list = fig.axes
    
    for idx, gsyn in enumerate(['gsyn_exc', 'gsyn_inh']):
        im = axes_list[idx].imshow(results[layer, gsyn].T)
        axes_list[idx].set_title(str(gsyn))
        axes_list[idx].set_xlabel('time [ms]')
        axes_list[idx].set_ylabel('cells')
        fig.colorbar(im, ax=axes_list[idx], fraction=0.010, label='[uS]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# task 1: on the stimulus

- go to the cell '# make the thalamic input sources' and design and test a new type of thalamic input (e.g., more/sparser input cells, etc)
- find the critical limit of sequential spikes to get the network spiking